In [1]:
# !pip install umap-learn

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import imageio
from tensorflow.keras.initializers import he_normal
import datetime
import pandas as pd
from os import listdir
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler, LabelEncoder
from PIL import Image
import glob
from sklearn.cluster import KMeans
# import tensorflow_addons as tfa
import pickle
from sklearn.metrics import silhouette_samples, silhouette_score
import os
from sklearn.model_selection import train_test_split

# import umap

In [4]:
batch_size = 64
num_channels = 3
image_size = 64
latent_dim = 128
num_features = 1
root = '/content/drive/MyDrive/CancerCell/24 April/NewDataSet_Ravaee_Feb2024/Dataset1 Scripts/'
images_root = root + 'images/'

In [5]:
df = pd.read_csv(root + 'features_JM28_3Lines_DNA_H2AX_ki67_20240401.csv')
data = df.iloc[:,11:].values
data = StandardScaler().fit_transform(data)

In [6]:
# _umap = umap.UMAP(random_state=123)
# embbedings_data = _umap.fit_transform(data)
# plt.scatter(embbedings_data[:,0], embbedings_data[:,1],s=1)
# plt.show()

In [7]:
# kmeans = KMeans(n_clusters=3, random_state=0, n_init="auto").fit(embbedings_data)
# labels = kmeans.predict(embbedings_data)
# centroids = kmeans.cluster_centers_
# distances = [np.linalg.norm(x-centroids[labels[i]]) for i,x in enumerate(embbedings_data)]

# threshold = 2.5*np.std(distances)

# data = data[np.where(distances < threshold)]
# clusters = kmeans.labels_[np.where(distances < threshold)]
# embbedings_data = embbedings_data[np.where(distances < threshold)]
# print(data.shape)

# plt.scatter(embbedings_data[:,0], embbedings_data[:,1],s=1)
# plt.show()

In [8]:
# pickle.dump(kmeans, open(root+"kmeans.pickle", "wb"))

# pickle.dump(_umap, open(root+"umap.pickle", "wb"))

In [9]:
# df = df[distances < threshold]

# clusters = (clusters / 3) - 0.5
# df['clusters'] = clusters
# print(np.unique(clusters))

In [10]:
# df.shape

In [11]:
img_files = os.listdir(images_root)
print(len(img_files))

20656


In [12]:
df['FileName_DNA'] =  df['FileName_DNA'].str.replace('.tif','') + "_FilterNuclei_" + df['ObjectNumber'].astype(str) +'.png'

In [13]:
df = df[df['FileName_DNA'].isin(img_files)]
df['FileName_DNA'] = images_root + df['FileName_DNA']
print(len(df))

20656


<ipython-input-13-90005a441244>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['FileName_DNA'] = images_root + df['FileName_DNA']


In [14]:
num_classes = np.unique(df['Metadata_Lines'].values).shape[0]

In [15]:
y_labels = LabelEncoder().fit_transform(df['Metadata_Lines'].values)
y_labels = tf.keras.utils.to_categorical(y_labels , 3)
x_train, x_val, y_train, y_val = train_test_split(df['FileName_DNA'].values ,y_labels, test_size=0.3 )

In [16]:
def map_fn(path, c):
    image = tf.image.decode_png(tf.io.read_file(path),channels=num_channels)
    #image = tfa.image.equalize(image)
    image = tf.image.resize_with_crop_or_pad(image, target_height=image_size,target_width=image_size)
    image = (tf.cast(image, tf.float32)) / 255.0
    return image, c

In [17]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.map(map_fn)
train_ds = (train_ds
    .shuffle(1024)
    .cache()
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

In [18]:
for x,y in train_ds.take(1):
  print(x.shape,y.shape)

(64, 64, 64, 3) (64, 3)


In [19]:
test_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test_ds = test_ds.map(map_fn)
test_ds = (test_ds
    .cache()
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

In [20]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10 ,restore_best_weights=True)

In [21]:
model = Sequential()

model.add(Conv2D(32 , kernel_size=3 , padding="same" , input_shape=(image_size,image_size,num_channels)))
model.add(LeakyReLU(alpha=0.2))


model.add(Conv2D(64 , kernel_size=3 , padding="same" ))
model.add(LeakyReLU(alpha=0.2))

model.add(Conv2D(64 , kernel_size=3 , strides=2 , padding="same" ))
model.add(LeakyReLU(alpha=0.2))

model.add(Conv2D(128 , kernel_size=3 , strides=2 , padding="same" ))
model.add(LeakyReLU(alpha=0.2))

model.add(Conv2D(128 , kernel_size=3 , strides=2 , padding="same" ))
model.add(LeakyReLU(alpha=0.2))

model.add(Conv2D(256 , kernel_size=3 , strides=2 , padding="same" ))
model.add(LeakyReLU(alpha=0.2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# softmax classifier
model.add(Dense(num_classes))
model.add(Activation("softmax"))

In [22]:
opt = Adam(learning_rate=1e-4)

model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 64, 64, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 64, 64, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 64)        36928     
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 32, 32, 64)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 128)       7

In [23]:
hist = model.fit(train_ds, epochs=1000, validation_data=test_ds, callbacks=[stop_early])

Epoch 1/1000
226/226 [==============================] - 77s 289ms/step - loss: 0.6180 - accuracy: 0.7604 - val_loss: 0.4422 - val_accuracy: 0.8364
Epoch 2/1000
226/226 [==============================] - 9s 38ms/step - loss: 0.4329 - accuracy: 0.8407 - val_loss: 0.4150 - val_accuracy: 0.8404
Epoch 3/1000
226/226 [==============================] - 9s 38ms/step - loss: 0.3869 - accuracy: 0.8571 - val_loss: 0.3750 - val_accuracy: 0.8570
Epoch 4/1000
226/226 [==============================] - 8s 37ms/step - loss: 0.3646 - accuracy: 0.8652 - val_loss: 0.3648 - val_accuracy: 0.8583
Epoch 5/1000
226/226 [==============================] - 9s 38ms/step - loss: 0.3478 - accuracy: 0.8749 - val_loss: 0.3418 - val_accuracy: 0.8683
Epoch 6/1000
226/226 [==============================] - 9s 38ms/step - loss: 0.3280 - accuracy: 0.8828 - val_loss: 0.3198 - val_accuracy: 0.8806
Epoch 7/1000
226/226 [==============================] - 9s 38ms/step - loss: 0.3179 - accuracy: 0.8851 - val_loss: 0.3186 - val_